In [1]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.nn as nn
import random
import numpy as np
from torchvision.ops import deform_conv2d
from torchvision.ops import DeformConv2d
device = torch.device('cuda')
from torchvision.transforms import v2
from torch.utils.data import default_collate

### Import Fashion MNIST Dataset

In [2]:
# Download and load the MNIST dataset
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform= ToTensor())
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=ToTensor())

### Set Seed

In [3]:
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

### Dataloaders

In [4]:
batch_size = 50
cutmix = v2.CutMix(num_classes=10)
mixup = v2.MixUp(num_classes=10)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])
def collate_fn(batch):
    return cutmix_or_mixup(*default_collate(batch))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size= batch_size, shuffle=False)

### Define LeNet5 CNN with Residual Block architecture

In [5]:
# Define a simple CNN model
class LeNet5(nn.Module):
    def __init__(self, input_shape=(1, 28, 28), num_classes=10):
        super(LeNet5, self).__init__()
        # self.offset1 = None
        # self.offset2 = None
        self.offset1 = nn.Parameter(torch.zeros(batch_size, 2*5*5, 24,24, requires_grad=True))
        self.offset2 = nn.Parameter(torch.zeros(batch_size, 2*5*5, 8,8, requires_grad=True))
        self.mask1 = nn.Parameter(torch.zeros(batch_size, 2*5*5, 24,24, requires_grad=True))
        self.mask2 = nn.Parameter(torch.zeros(batch_size, 2*5*5, 24,24, requires_grad=True))
        self.deformconv1 = DeformConv2d(1, 32, kernel_size=5, stride=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = DeformConv2d(32, 64, kernel_size=5, stride=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.classifier = nn.Sequential(
            nn.Linear(64 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        # if self.offset1 is None or self.offset1.shape[0] != x.shape[0]:
        #     self.offset1 = nn.Parameter(torch.zeros(x.shape[0], 2 * 5 * 5, 24, 24))
        # if self.offset2 is None or self.offset2.shape[0] != x.shape[0]:
        #     self.offset2 = nn.Parameter(torch.zeros(x.shape[0], 2 * 5 * 5, 8, 8))
        x = self.deformconv1(x, self.offset1)
        x = self.batchnorm1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x, self.offset2)
        x = self.batchnorm2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [6]:
loss_fn = nn.CrossEntropyLoss()
device = torch.device('cuda')
model = LeNet5().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Train Function

In [7]:
def train(train_loader, model, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        # Calculate average training loss for the epoch
        avg_train_loss = train_loss / len(train_dataloader)
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}')

### Test Function

In [8]:
def test(test_loader, model):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            loss = loss_fn(output, labels)
            test_loss += loss.item()

            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy for the epoch
    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = 100 * (correct / total)

    return avg_test_loss, accuracy

### Commence Training

In [9]:
train(train_dataloader, model, 50)

Epoch 1/50, Train Loss: 1.8850
Epoch 2/50, Train Loss: 1.8357
Epoch 3/50, Train Loss: 1.8212
Epoch 4/50, Train Loss: 1.8232
Epoch 5/50, Train Loss: 1.8165
Epoch 6/50, Train Loss: 1.8068
Epoch 7/50, Train Loss: 1.8139
Epoch 8/50, Train Loss: 1.8128
Epoch 9/50, Train Loss: 1.8040
Epoch 10/50, Train Loss: 1.8097
Epoch 11/50, Train Loss: 1.7964
Epoch 12/50, Train Loss: 1.8079
Epoch 13/50, Train Loss: 1.8023
Epoch 14/50, Train Loss: 1.7963
Epoch 15/50, Train Loss: 1.7956
Epoch 16/50, Train Loss: 1.8033
Epoch 17/50, Train Loss: 1.8030
Epoch 18/50, Train Loss: 1.7951
Epoch 19/50, Train Loss: 1.7905
Epoch 20/50, Train Loss: 1.8001
Epoch 21/50, Train Loss: 1.7909
Epoch 22/50, Train Loss: 1.7906
Epoch 23/50, Train Loss: 1.7973
Epoch 24/50, Train Loss: 1.7973
Epoch 25/50, Train Loss: 1.7907
Epoch 26/50, Train Loss: 1.7853
Epoch 27/50, Train Loss: 1.8008
Epoch 28/50, Train Loss: 1.7973
Epoch 29/50, Train Loss: 1.7939
Epoch 30/50, Train Loss: 1.7919
Epoch 31/50, Train Loss: 1.7914
Epoch 32/50, Trai

### Test trained model on unseen data

In [10]:
avg_test_loss, acc = test(test_dataloader, model)

In [11]:
print("Accuracy on base model: " + str(acc) + "%")

Accuracy on base model: 88.79%
